In [8]:
import cobra
from cobra.io import load_model
import escher
from escher import Builder
from time import sleep
import pandas
from cobra import Model, Reaction, Metabolite

In [11]:
#define model
model = load_model("iJO1366")
#Run FBA, see growth rate under normal import circumstances
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')

Growth rate: 0.9823718127269633 1/h


In [ ]:
#what if we decrease 'amount' of glucose in medium (i.e. decrease import rate of glucose? what happens to growth rate?)
model.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000

model.reactions.get_by_id("EX_glc__D_e").lower_bound = -10
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')


In [ ]:
#convert fluxes to df
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass.csv')

In [4]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [6]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='iJO1366',
)
builder

Builder()

In [ ]:
#simulate deletions
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
with model:
    model.reactions.PGI.knock_out()
    print('PGI knocked out: ', model.optimize())
    solution = model.optimize()
    print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
    df = pandas.DataFrame.from_dict([solution.fluxes]).T
    df.to_csv('FBA_max_biomas1.csv')  

In [13]:
#add heterologous reactions for genes crtE, crtB, crtI
#define pre-existing metabolites
ipdp_c = model.metabolites.get_by_id("ipdp_c")
frdp_c = model.metabolites.get_by_id("frdp_c")
ppi_c = model.metabolites.get_by_id("ppi_c")
nadp_c = model.metabolites.get_by_id("nadp_c")
nadph_c = model.metabolites.get_by_id("nadph_c")
#create new metabolites
ggpp_c = Metabolite('ggpp_c',formula ='C20H36O7P2',name ='Geranylgeranyl pyrophosphate', compartment='c')
phyto_c = Metabolite('phyto_c',formula ='C40H64', name ='Phytoene', compartment ='c')
lyco_c = Metabolite('lyco_c',formula ='C40H56', name ='Lycopene', compartment ='c')
#create new reactions
reaction1 = Reaction('crtE')
reaction1.name = 'geranylgeranyl pyrophosphate synthase (crtE)'
reaction1.subsystem = 'Lycopene Biosynthesis'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
reaction1.add_metabolites({ipdp_c: -1.0, frdp_c:-1.0,ggpp_c:1.0,ppi_c:1.0})
model.add_reactions(reaction1)
reaction2 = Reaction('crtB')
reaction2.name = 'phytoene synthase (crtB)'
reaction2.subsystem = 'Lycopene Biosynthesis'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
reaction2.add_metabolites({ggpp_c:-2.0,phyto_c:1.0,ppi_c:1.0})
model.add_reactions(reaction2)
reaction3 = Reaction('crtI')
reaction3.name = 'phytoene desaturase (crtI)'
reaction3.subsystem = 'Lycopene Biosynthesis'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({phyto_c:-1.0,nadp_c:-8.0,lyco_c:1.0,nadph_c:1.0})
model.add_reactions(reaction3)
reaction4 = Reaction('LYCO-dem')
reaction4.name = 'Lycopene demand'
reaction4.subsystem = 'Lycopene biosynthesis'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({lyco_c: -1.0})
model.add_reactions(reaction4)

TypeError: 'Reaction' object is not iterable